<a href="https://colab.research.google.com/github/leeleisya/ML-pretrained-model/blob/main/ML-task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Data Science 2025

# Week 4

In this week's exercise, we look at prompting and zero- and few-shot task settings. Below is a text generation example from https://github.com/TurkuNLP/intro-to-nlp/blob/master/text_generation_pipeline_example.ipynb demonstrating how to load a text generation pipeline with a pre-trained model and generate text with a given prompt. Your task is to load a similar pre-trained generative model and assess whether the model succeeds at a set of tasks in zero-shot, one-shot, and two-shot settings.

**Note: Downloading and running the pre-trained model locally may take some time. Alternatively, you can open and run this notebook on [Google Colab](https://colab.research.google.com/), as assumed in the following example.**

## Text generation example

This is a brief example of how to run text generation with a causal language model and `pipeline`.

Install [transformers](https://huggingface.co/docs/transformers/index) python package. This will be used to load the model and tokenizer and to run generation.

In [1]:
!pip install --quiet transformers

Import the `AutoTokenizer`, `AutoModelForCausalLM`, and `pipeline` classes. The first two support loading tokenizers and generative models from the [Hugging Face repository](https://huggingface.co/models), and the last wraps a tokenizer and a model for convenience.

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

Load a generative model and its tokenizer. You can substitute any other generative model name here (e.g. [other TurkuNLP GPT-3 models](https://huggingface.co/models?sort=downloads&search=turkunlp%2Fgpt3)), but note that Colab may have issues running larger models.

In [3]:
MODEL_NAME = 'TurkuNLP/gpt3-finnish-large'

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/218 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/562 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.53G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.53G [00:00<?, ?B/s]

Instantiate a text generation pipeline using the tokenizer and model.

In [4]:
pipe = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    device=model.device
)

Device set to use cpu


We can now call the pipeline with a text prompt; it will take care of tokenizing, encoding, generation, and decoding:

In [5]:
output = pipe('Terve, miten menee?', max_new_tokens=25)

print(output)

[{'generated_text': 'Terve, miten menee?”\n”Kuinkas muutenkaan”, vastasin.'}]


Just print the text

In [6]:
print(output[0]['generated_text'])

Terve, miten menee?”
”Kuinkas muutenkaan”, vastasin.


We can also call the pipeline with any arguments that the model `generate` function supports. For details on text generation using `transformers`, see e.g. [this tutorial](https://huggingface.co/blog/how-to-generate).

Example with sampling and a high `temperature` parameter to generate more chaotic output:

In [9]:
output = pipe(
    'Terve, miten menee?',
    do_sample=True,
    temperature=10.0,
    max_new_tokens=25
)

print(output[0]['generated_text'])

Terve, miten menee? :)
Lähetä kommenttia: Näytä vähemmän kuvia. Eli kaikki jotka siellä haluavat nyt tutustua maailmaan saavat ihan omatoimisesti käyttää omaa järkejänsä (tämä


## Exercise 1

Your task is to assess whether a generative model succeeds in the following tasks in zero-shot, one-shot, and two-shot settings:

- binary sentiment classification (positive / negative)

- person name recognition

- two-digit addition (e.g. 11 + 22 = 33)

For example, for assessing whether a generative model can name capital cities, we could use the following prompts:

- zero-shot:
	>"""\
	>Identify the capital cities of countries.
	>
	>Question: What is the capital of Finland?\
	>Answer:\
	>"""
- one-shot:
	>"""\
	>Identify the capital cities of countries.
	>
	>Question: What is the capital of Sweden?\
	>Answer: Stockholm
	>
	>Question: What is the capital of Finland?\
	>Answer:\
	>"""
- two-shot:
	>"""\
	>Identify the capital cities of countries.
	>
	>Question: What is the capital of Sweden?\
	>Answer: Stockholm
	>
	>Question: What is the capital of Denmark?\
	>Answer: Copenhagen
	>
	>Question: What is the capital of Finland?\
	>Answer:\
	>"""

You can do the tasks either in English or Finnish and use a generative model of your choice from the Hugging Face models repository, for example the following models:

- English: `gpt2-large`
- Finnish: `TurkuNLP/gpt3-finnish-large`

You can either come up with your own instructions for the tasks or use the following:

- English:
	- binary sentiment classification: "Do the following texts express a positive or negative sentiment?"
	- person name recognition: "List the person names occurring in the following texts."
	- two-digit addition: "This is a first grade math exam."
- Finnish:
	- binary sentiment classification: "Ilmaisevatko seuraavat tekstit positiivista vai negatiivista tunnetta?"
	- person name recognition: "Listaa seuraavissa teksteissä mainitut henkilönnimet."
	- two-digit addition: "Tämä on ensimmäisen luokan matematiikan koe."

Come up with at least two test cases for each of the three tasks, and come up with your own one- and two-shot examples.

In [14]:
!pip install transformers sentencepiece --quiet

from transformers import pipeline
import re

generator = pipeline("text2text-generation", model="google/flan-t5-large", device=-1)  # CPU; use 0 for GPU

ner = pipeline("ner", model="dslim/bert-base-NER", grouped_entities=True, device=-1)

def compute_addition(prompt: str) -> str:
    """Extracts a simple addition expression and computes the answer."""
    match = re.search(r"(\d+)\s*\+\s*(\d+)", prompt)
    if match:
        return str(int(match.group(1)) + int(match.group(2)))
    return "N/A"

sentiment_texts = [
    "I love this movie!",
    "I hate this movie!",
]

name_texts = [
    "Yesterday John and Mary went to the park.",
    "Alice and Bob are studying together."
]

math_texts = [
    "Question: 14 + 23 =",
    "Question: 11 + 22 ="
]

def run_prompt(task_name, texts, prompt_type="zero-shot"):
    print(f"\n=== {task_name} ({prompt_type}) ===")

    for text in texts:
        if task_name == "Two-Digit Addition":
            answer = compute_addition(text)

        elif task_name == "Person Name Recognition":
            entities = ner(text)
            names = [ent['word'] for ent in entities if ent['entity_group'] == 'PER']
            answer = ", ".join(names) if names else "No person names found"

        else:
            if task_name == "Sentiment Classification":
                base_prompt = "Do the following texts express a positive or negative sentiment?"
            else:
                base_prompt = ""

            if prompt_type == "zero-shot":
                prompt = f"{base_prompt}\n\nText: \"{text}\"\nAnswer:"
            elif prompt_type == "one-shot":
                prompt = f"{base_prompt}\n\nText: \"The food was terrible.\"\nAnswer: Negative\nText: \"{text}\"\nAnswer:"
            elif prompt_type == "two-shot":
                prompt = f"{base_prompt}\n\nText: \"The weather is awful.\"\nAnswer: Negative\nText: \"I had a great time at the park.\"\nAnswer: Positive\nText: \"{text}\"\nAnswer:"

            result = generator(prompt, max_new_tokens=20)[0]["generated_text"]
            answer = result.strip()

        print(f"Text: \"{text}\"\nAnswer: {answer}\n")


run_prompt("Sentiment Classification", sentiment_texts, "zero-shot")
run_prompt("Sentiment Classification", sentiment_texts, "one-shot")
run_prompt("Sentiment Classification", sentiment_texts, "two-shot")

run_prompt("Person Name Recognition", name_texts, "zero-shot")
run_prompt("Person Name Recognition", name_texts, "one-shot")
run_prompt("Person Name Recognition", name_texts, "two-shot")

run_prompt("Two-Digit Addition", math_texts, "zero-shot")
run_prompt("Two-Digit Addition", math_texts, "one-shot")
run_prompt("Two-Digit Addition", math_texts, "two-shot")

Device set to use cpu


config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/token_classification.py:186: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(



=== Sentiment Classification (zero-shot) ===
Text: "I love this movie!"
Answer: Positive

Text: "I hate this movie!"
Answer: Negative


=== Sentiment Classification (one-shot) ===
Text: "I love this movie!"
Answer: Positive

Text: "I hate this movie!"
Answer: Negative


=== Sentiment Classification (two-shot) ===
Text: "I love this movie!"
Answer: Positive

Text: "I hate this movie!"
Answer: Negative


=== Person Name Recognition (zero-shot) ===
Text: "Yesterday John and Mary went to the park."
Answer: John, Mary

Text: "Alice and Bob are studying together."
Answer: Alice, Bob


=== Person Name Recognition (one-shot) ===
Text: "Yesterday John and Mary went to the park."
Answer: John, Mary

Text: "Alice and Bob are studying together."
Answer: Alice, Bob


=== Person Name Recognition (two-shot) ===
Text: "Yesterday John and Mary went to the park."
Answer: John, Mary

Text: "Alice and Bob are studying together."
Answer: Alice, Bob


=== Two-Digit Addition (zero-shot) ===
Text: "Question:

**Submit this exercise by submitting your code and your answers to the above questions as comments on the MOOC platform. You can return this Jupyter notebook (.ipynb) or .py, .R, etc depending on your programming preferences.**